In [1]:
!pip install pymupdf easyocr opencv-python pillow # easy ocr
!pip install pymupdf4llm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 54.4 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.4 MB/s eta 0:00:00:00:0100:01
  Attempting u

In [2]:
import io
import os
import re
import cv2
import fitz  # PyMuPDF
import gdown
import torch
import zipfile
import pickle
import easyocr
import pymupdf4llm
import pandas as pd
import numpy as np
from PIL import Image

from langchain.schema import Document

### 기본 Utils (basic_utils.py)

In [3]:
def load_secret(key_name: str):

    # 1) Kaggle 환경: os.environ 또는 파일에서 읽기
    if key_name in os.environ:
        return os.environ[key_name]

    try:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        return user_secrets.get_secret(key_name)
    except ImportError:
        pass

    # 2) Colab 환경인지 확인
    try:
        import google.colab.userdata as userdata
        try:
            return userdata.get(key_name)
        except KeyError:
            pass # Key not in Colab userdata
    except ImportError:
        pass

    raise KeyError(f"Secret '{key_name}' not found in Colab userdata, os.environ, or file.")


def download(id, filename):
    if os.path.isfile(filename) and os.path.getsize(filename) > 0:
        print(f"[skip] {filename} 이미 존재합니다.")
    else:
        gdown.download(id=id, output=filename, quiet=False)
        print(f'[ok] {filename} 다운로드 완료.')

        if filename.lower().endswith('.zip'):
            try:
                with zipfile.ZipFile(filename, 'r') as zip_ref:
                    zip_ref.extractall(os.path.dirname(filename) or '.') # Extract to directory of the file, or current if no directory
                print(f'[ok] {filename} 압축풀기 완료.')
            except zipfile.BadZipFile:
                print(f"[warning] {filename}은 유효한 zip 파일이 아닙니다.")

    return filename


def load_csv(filename):
  """Loads a CSV file into a pandas DataFrame."""
  try:
    df = pd.read_csv(filename)
    print(f"[ok] {filename} 로드 완료.")
    return df
  except FileNotFoundError:
    print(f"[error] {filename} 파일을 찾을 수 없습니다.")
    return None
  except Exception as e:
    print(f"[error] {filename} 로드 중 오류 발생: {e}")
    return None


def save_pages(pages, filename):
    directory = os.path.dirname(filename)
    if directory and not os.path.exists(directory):
        os.makedirs(directory)
        print(f"디렉토리 '{directory}'를 생성했습니다.")
    with open(filename, 'wb') as f:
        pickle.dump(pages, f)
    print(f"'{filename}' 파일에 pages를 저장했습니다.")


def load_pages(filename):
    if not os.path.exists(filename):
        print(f"오류: '{filename}' 파일을 찾을 수 없습니다.")
        return None
    with open(filename, 'rb') as f:
        pages = pickle.load(f)
    print(f"'{filename}' 파일에서 pages를 로드했습니다.")
    return pages

def ext(original_filename, ext='pkl'):
  base_filename, _ = os.path.splitext(original_filename)
  return f"{base_filename}.{ext}"

### PDF/HWP 문서 Utils (document_utils.py)

In [ ]:
def load_pymupdf(pdf_path, filename, langs=['ko','en'], zoom=3.0, gpu=True):
    """
    PDF 페이지별 텍스트 레이어 + EasyOCR 결과 병합
    - langs: EasyOCR 언어 리스트
    - zoom: 해상도 배율 (3.0이면 약 216dpi)
    - gpu: GPU 사용 여부
    """
    # 캐시 로드
    output_path = f'outputs/{filename}'
    if os.path.exists(output_path):
        return load_pages(output_path)

    # EasyOCR Reader 초기화
    reader = easyocr.Reader(langs, gpu=gpu)

    doc = fitz.open(f'files/{pdf_path}')
    pages = []

    for page_num, page in enumerate(doc, start=1):
        # 1. 텍스트 레이어 추출
        text_layer = page.get_text()

        # 2. 고해상도 렌더링
        matrix = fitz.Matrix(zoom, zoom)
        pix = page.get_pixmap(matrix=matrix, alpha=False)
        img = Image.open(io.BytesIO(pix.tobytes("png")))

        # 3. EasyOCR 실행
        cv_img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        results = reader.readtext(cv_img, detail=0)  # detail=0 → 텍스트만 리스트로 반환
        ocr_text = "\n".join(results)

        # 4. 병합
        merged_text = (text_layer or "").strip() + "\n" + (ocr_text or "").strip()

        print(f"[페이지 {page_num}] 텍스트 길이: {len(text_layer)}, OCR 길이: {len(ocr_text)}")
        pages.append(Document(page_content=merged_text))
        display(img)
        print(ocr_text)
    doc.close()

    # 캐시 저장
    save_pages(pages, output_path)
    return pages


def load_pymupdf4llm_easyocr(pdf_path, output_path):
    # EasyOCR 리더 객체 초기화 (GPU 사용 가능 시 활용)
    IMAGE_PATH = 'images'
    os.makedirs(IMAGE_PATH, exist_ok=True)
    reader = easyocr.Reader(['ko', 'en'], gpu=torch.cuda.is_available())

    # 1. PyMuPDF4LLM을 사용해 마크다운 생성 및 이미지 추출
    # write_images=True 옵션으로 이미지 추출
    # image_path 옵션으로 이미지 저장 위치 지정
    # page_chunks=True 옵션으로 페이지별로 분리된 결과 생성
    extracted_chunks = pymupdf4llm.to_markdown(pdf_path, write_images=True, image_path=IMAGE_PATH, page_chunks=True)

    final_markdown = ""
    
    # 2. EasyOCR로 이미지 내 텍스트 인식 후 마크다운에 추가
    for chunk in extracted_chunks:
        page_text = chunk["text"]
        
        # 마크다운 내 이미지 참조 찾기 (예: ![image_name](images/image_name.png))
        # ([^\]]+)는 이미지 이름 부분을 캡처
        image_ref_pattern = re.compile(r"!\[(.*?)\]\((.*?)\)")
        matches = image_ref_pattern.finditer(page_text)
        
        # 이미지 참조를 순서대로 처리
        for match in reversed(list(matches)):
            image_path = match.group(2)
            full_image_path = os.path.join(os.path.dirname(output_path), image_path)
            
            # EasyOCR로 이미지 내 텍스트 인식
            try:
                ocr_results = reader.readtext(full_image_path)
                ocr_caption = " ".join([text for _, text, _ in ocr_results])
            except Exception as e:
                print(f"OCR 처리 중 오류 발생: {e}")
                ocr_caption = "OCR 처리 실패"
            
            # 마크다운 텍스트 업데이트 (이미지 참조 다음에 OCR 결과 삽입)
            original_match_string = match.group(0)
            replacement_string = f"{original_match_string}\n\n**OCR 텍스트:** {ocr_caption}\n"
            page_text = page_text[:match.start()] + replacement_string + page_text[match.end():]
        
        final_markdown += page_text + "\n\n"

    return final_markdown

### 필요자료 다운로드

In [11]:
download('1t9TWN25lsshk_tIXyh3Gx-NzfNAWRdeb', 'output.zip')

[skip] output.zip 이미 존재합니다.


'output.zip'

### CSV 파일 확인하기

In [ ]:
df = load_csv('data_list.csv')
df.head()

### PDF 샘플 로딩

In [ ]:
pdf_df = df[df['파일형식'] == 'pdf']
pdf_name = pdf_df.iloc[0]['파일명'] # 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf
pdf_path = f'files/{pdf_name}'
output_path = f'outputs/{ext(pdf_name, "md")}'
markdown_output = load_pymupdf4llm_easyocr(pdf_path, output_path) # 추후 pages를 반환하도록 변경해야 함.
# markdown_output = pages.join('\n\n---\n\n')
with open(output_path, "w", encoding="utf-8") as f:
    f.write(markdown_output)
print(f"PDF 처리 완료. {output_path} 파일 생성.")
with open(ext(pdf_name, 'md'), "w", encoding="utf-8") as f:
    f.write(markdown_output)

### HWP 샘플 로딩

In [ ]:
# 예시 사용
hwp_name = df.iloc[0]['파일명']
print(f"원본 파일명: {hwp_name}")
print(f".pkl 파일명: {ext(hwp_name)}")